In [1]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

TYPE_MAP = {0:'vert',
            1:'horz'}

settings.MUTATION_PRO = 0.01


#increasing env setting

env_settings.inc_env = {
    1: [0.5 + (i * 1e-3) for i in range(4000)] + [0.5 + (3999 * 1e-3)] * 2000,
}

class A_3(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    
        
class M_3(GenModel):
    def get_agent_config(self):
        return A_3
    
    def init_env(self):
        #no influence
        return env_settings.inc_env[self.env_val]
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        self.model_name = 'M3'
        
        init_ratio = specific_kargs['init_ratio']
        self.env_val = specific_kargs['env_val']
        
        
        settings.INIT_MEAN = specific_kargs['gene_mean']
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_SD_LOW = -1 * 2 * settings.INIT_SD
        settings.INIT_SD_HIGH = 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = specific_kargs['muta_sd']
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([str(i) for i in (init_ratio, settings.INIT_SD ,settings.POPU_MUTA_SD)])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = check_path("./res/")
        data_file_name = check_path(data_file_name + self.model_name)
        data_file_name = check_path(data_file_name + "/" + self.specific_kargs_str)
        data_file_name = data_file_name + '/' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(
            ('step',
             TYPE_MAP[0], TYPE_MAP[0] + ' beta', TYPE_MAP[0] + 'gene', 'var',  
             TYPE_MAP[1], TYPE_MAP[1] + ' beta', TYPE_MAP[1] + 'gene', 'var'))+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_gene_mean(self, gen_type):
        return [np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type]
    
    def get_popu_gene_mean(self, gene_mean):
        return np.mean(gene_mean)

    def get_popu_gene_var(self, gene_mean):
        return np.var(gene_mean)

    def collect_data(self):
        ver_mean = self.get_gene_mean(0)
        hor_mean = self.get_gene_mean(1)
        return (self.schedule.steps,
                self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(ver_mean), self.get_popu_gene_var(ver_mean),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(hor_mean), self.get_popu_gene_var(hor_mean))
        
        
    def step(self):
        data = self.collect_data()
        #print(data)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        if self.schedule.steps % 100 == 0: 
            self.data_file.flush()
        super().step()
        
        


In [2]:
fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': [{'env_val': 1 , 'init_ratio':0.5, 'gene_mean':0.5, 'gene_sd':0.1, 'muta_sd':0.01}]}

batch_run = BatchRunnerMP(
    M_3, 2,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=10,
    max_steps=800,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./res/M3/0.5_0.1_0.01/3a8334c6-ce51-11e9-a222-f2189850a77c
data in file  ./res/M3/0.5_0.1_0.01/3a83c396-ce51-11e9-8414-f2189850a77c
0 population size:  2000
0 population size:  2000
1 population size:  1483
1 population size:  1514
2 population size:  1158
2 population size:  1208
3 population size:  993
3 population size:  984
4 population size:  853
4 population size:  856
5 population size:  777
5 population size:  775
6 population size:  750
6 population size:  756
7 population size:  769
7 population size:  733
8 population size:  813
8 population size:  764
9 population size:  916
9 population size:  827
10 population size:  1067
10 population size:  931
11 population size:  1122
11 population size:  1295
12 population size:  1378
12 population size:  1587


Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-packages/multiprocess/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.7/site-packages/multiprocess/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.7/site-packages/pathos/helpers/mp_helper.py", line 15, in <lambda>
    func = lambda args: 

data in file  ./res/M3/0.5_0.1_0.01/41e189f2-ce51-11e9-92b2-f2189850a77c
data in file  ./res/M3/0.5_0.1_0.01/41e29914-ce51-11e9-879b-f2189850a77c


KeyboardInterrupt: 

0 population size:  2000
0 population size:  2000
1 population size:  1427
1 population size:  1468
2 population size:  1148
2 population size:  1120
